In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nose.tools import *

np.random.seed(24680)

Write your imports in the cell below.

In [26]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.metrics import f1_score

# Ensemble Models and Support Vector Machines Lab
## Training and comparing different algorithms

Once again, we'll work with the bank dataset. This time, the data preprocessing steps have been done for you.

The goal is to try and improve our predictions (if they can be improved at all) using different types of algorithms.

### 1. Read the data (1 point)
This time you only need to read the data. The indicator variables have been separated out for you.

Read the dataset and save it in the variable `bank_data`. The target column is `y`. Use the variables `bank_attributes` and `bank_labels` to save the attributes (explanatory variables, features, predictors), and labels (`y`).

In [10]:
bank_data = pd.read_csv("./data/bank.csv")
bank_data

age  balance  day  duration  campaign  pdays  previous  job_admin.  \
0      30     1787   19        79         1     -1         0           0   
1      33     4789   11       220         1    339         4           0   
2      35     1350   16       185         1    330         1           0   
3      30     1476    3       199         4     -1         0           0   
4      59        0    5       226         1     -1         0           0   
...   ...      ...  ...       ...       ...    ...       ...         ...   
4516   33     -333   30       329         5     -1         0           0   
4517   57    -3313    9       153         1     -1         0           0   
4518   57      295   19       151        11     -1         0           0   
4519   28     1137    6       129         4    211         3           0   
4520   44     1136    3       345         2    249         7           0   

      job_blue-collar  job_entrepreneur  ...  month_mar  month_may  month_nov  \
0                   0                 0  ...          0          0          0   
1                   0                 0  ...          0          1          0   
2                   0                 0  ...          0          0          0   
3                   0                 0  ...          0          0          0   
4                   1                 0  ...          0          1          0   
...               ...               ...  ...        ...        ...        ...   
4516                0                 0  ...          0          0          0   
4517                0                 0  ...          0          1          0   
4518                0                 0  ...          0          0          0   
4519                1                 0  ...          0          0          0   
4520                0                 1  ...          0          0          0   

      month_oct  month_sep  poutcome_failure  poutcome_other  \
0             1          0                 0               0   
1             0          0                 1               0   
2             0          0                 1               0   
3             0          0                 0               0   
4             0          0                 0               0   
...         ...        ...               ...             ...   
4516          0          0                 0               0   
4517          0          0                 0               0   
4518          0          0                 0               0   
4519          0          0                 0               1   
4520          0          0                 0               1   

      poutcome_success  poutcome_unknown  y  
0                    0                 1  0  
1                    0                 0  0  
2                    0                 0  0  
3                    0                 1  0  
4                    0                 1  0  
...                ...               ... ..  
4516                 0                 1  0  
4517                 0                 1  0  
4518                 0                 1  0  
4519                 0                 0  0  
4520                 0                 0  0  

[4521 rows x 52 columns]

In [11]:
bank_data = pd.read_csv("./data/bank.csv")
bank_attributes = bank_data.drop('y', axis=1)
bank_labels = bank_data.y
bank_attributes.head(10)

age  balance  day  duration  campaign  pdays  previous  job_admin.  \
0   30     1787   19        79         1     -1         0           0   
1   33     4789   11       220         1    339         4           0   
2   35     1350   16       185         1    330         1           0   
3   30     1476    3       199         4     -1         0           0   
4   59        0    5       226         1     -1         0           0   
5   35      747   23       141         2    176         3           0   
6   36      307   14       341         1    330         2           0   
7   39      147    6       151         2     -1         0           0   
8   41      221   14        57         2     -1         0           0   
9   43      -88   17       313         1    147         2           0   

   job_blue-collar  job_entrepreneur  ...  month_jun  month_mar  month_may  \
0                0                 0  ...          0          0          0   
1                0                 0  ...          0          0          1   
2                0                 0  ...          0          0          0   
3                0                 0  ...          1          0          0   
4                1                 0  ...          0          0          1   
5                0                 0  ...          0          0          0   
6                0                 0  ...          0          0          1   
7                0                 0  ...          0          0          1   
8                0                 1  ...          0          0          1   
9                0                 0  ...          0          0          0   

   month_nov  month_oct  month_sep  poutcome_failure  poutcome_other  \
0          0          1          0                 0               0   
1          0          0          0                 1               0   
2          0          0          0                 1               0   
3          0          0          0                 0               0   
4          0          0          0                 0               0   
5          0          0          0                 1               0   
6          0          0          0                 0               1   
7          0          0          0                 0               0   
8          0          0          0                 0               0   
9          0          0          0                 1               0   

   poutcome_success  poutcome_unknown  
0                 0                 1  
1                 0                 0  
2                 0                 0  
3                 0                 1  
4                 0                 1  
5                 0                 0  
6                 0                 0  
7                 0                 1  
8                 0                 1  
9                 0                 0  

[10 rows x 51 columns]

In [12]:
assert_is_not_none(bank_data)
assert_is_not_none(bank_attributes)
assert_is_not_none(bank_labels)

### 2. Normalize the data (1 point)
Because both forests and SVMs are sensitive to non-scaled data, we need to normalize our dataset first.

Rescale all columns in `bank_attributes` so they have mean = 0 and variance = 1. You can either look at the `sklearn` docs or do this yourself. When you're ready, overwrite the `bank_attributes` column. Make sure that you don't lose the column names in the process.

In [15]:
bank_attributes_normalized = scale(bank_attributes)

In [16]:
bank_attributes.columns

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'default_no', 'default_yes', 'housing_no',
       'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'poutcome_failure', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown'],
      dtype='object')

In [17]:
bank_attributes = pd.DataFrame(bank_attributes_normalized, columns = bank_attributes.columns)

In [18]:
assert_is_not_none(bank_attributes)

### 3. Split the data (1 point)
Use the standard 70% / 30% split. Since this is a classification problem, be sure to stratify the split according to the `bank_labels`.

In [19]:
bank_attributes_train, bank_attributes_test, bank_labels_train, bank_labels_test = train_test_split(bank_attributes, bank_labels, train_size = 0.7, test_size = 0.3, stratify = bank_labels)

In [20]:
bank_labels_train.shape

(3164,)

In [21]:
assert_is_not_none(bank_attributes_train)
assert_is_not_none(bank_labels_train)

assert_is_not_none(bank_attributes_test)
assert_is_not_none(bank_labels_test)

### 4. Prepare the cross-validation folds (1 point)
Use a stratified k-fold cross-validation split, with $k = 5$. Fit it to the train data. Save the trained cross-validator to the variable `k_fold`.

The data should already be shuffled. There's no need to shuffle it again.

In [22]:
k_fold = StratifiedKFold(n_splits=5)
k_fold = k_fold.get_n_splits(bank_attributes_train, bank_labels_train)

In [24]:
assert_is_not_none(k_fold)

### 5. Decision Tree (2 points)
Use cross-validation to train and optimize the hyperparameters for a decision tree classifier.

Use grid search with the following grid:
* `max_depth`: 1, 5, 7, 15, 20
* `min_samples_leaf`: 2, 5, 10, 12
* `max_leaf_nodes`: 5, 10, 20

Use the most appropriate scoring metric (remember that accuracy doesn't work in this case because the data is highly imbalanced; we need something which combines precision and recall). Use the cross-validation splits you just created.

Save the grid results in `grid_search`. Save the best classifier in `tree_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [28]:
tuned_params = [{
    "max_depth": [1, 5, 7, 15, 20],
    "min_samples_leaf": [2, 5, 10, 12],
    "max_leaf_nodes": [5, 10, 20]
}]
grid_search = GridSearchCV(DecisionTreeClassifier(), tuned_params, cv = k_fold, scoring = "f1")
grid_search.fit(bank_attributes_train, bank_labels_train)
tree_classifier = grid_search.best_estimator_

print("Decision tree; best score:", grid_search.best_score_)

Decision tree; best score: 0.5090152112059995


In [30]:
tree_classifier

DecisionTreeClassifier(max_depth=15, max_leaf_nodes=10, min_samples_leaf=2)

In [31]:
print(grid_search.best_params_)

{'max_depth': 15, 'max_leaf_nodes': 10, 'min_samples_leaf': 2}


In [29]:
assert_is_not_none(grid_search)
assert_is_not_none(tree_classifier)

### 5. Random Forest (1 point)
Use cross-validation to train and optimize the hyperparameters for a random forest classifier. Use the same technique as before.

Use the following grid:
* `n_estimators`: 100, 200, 300 
* `max_depth`: 20, 50, 100

Note that this grid is on the small side but this is mainly due to performance reasons. Also note that the training will take some time.

Save the grid results in `grid_search`. Save the best classifier in `forest_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

Due to the relatively slow training, we've chosen low values for the parameters. The performance of the random forest will be worse than the decision tree. This is not necessarily the case in general, it's due to the parameters we've chosen to try.

In [32]:
tuned_params = [{
    "n_estimators": [100, 200, 300],
    "max_depth": [20, 50, 100]
}]
grid_search = GridSearchCV(RandomForestClassifier(), tuned_params, cv = k_fold, scoring = 'f1')
grid_search.fit(bank_attributes_train, bank_labels_train)
forest_classifier = grid_search.best_estimator_

print("Random forest; best score:", grid_search.best_score_)

Random forest; best score: 0.39249980844504256


In [34]:
print(grid_search.best_params_)

{'max_depth': 50, 'n_estimators': 200}


In [33]:
assert_is_not_none(grid_search)
assert_is_not_none(forest_classifier)

### 6. Linear SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for a linear support vector machine. Use the same technique as before.

Use the following grid:
* `C`: 0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20

Note that we're choosing relatively small values for `C`. This is allowed because our data is normalized.

Save the grid results in `grid_search`. Save the best classifier in `linear_svm_classifier`. There are many ways to create a linear SVM classifier. Look up the `sklearn` docs to choose the fastest one (in terms of performance).

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [35]:
tuned_params = [{
    "C": [0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20]
}]
grid_search = GridSearchCV(LinearSVC(), tuned_params, scoring = 'f1', cv = k_fold)
grid_search.fit(bank_attributes_train, bank_labels_train)
linear_svm_classifier = grid_search.best_estimator_

print("Linear SVM; best score:", grid_search.best_score_)

/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, inc

/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, inc

Linear SVM; best score: 0.39279456206568036


/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [37]:
print("Linear SVM; best score:", grid_search.best_score_)

Linear SVM; best score: 0.39279456206568036


In [36]:
grid_search.best_params_

{'C': 6}

In [38]:
assert_is_not_none(grid_search)
assert_is_not_none(linear_svm_classifier)

### 7. Gaussian SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for an SVM with a Gaussian kernel. Use the same technique as before.

Use the following grid:
* `C`: 10, 15, 20, 50, 200
* `gamma`: 0.001, 0.01, 0.1, 0.2

Note that this time we give larger values of `C` because the governing parameter here is `gamma`.

Save the grid results in `grid_search`. Save the best classifier in `gaussian_svm_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [39]:
tuned_params = [{
    "C": [10, 15, 20, 50, 200],
    "gamma": [0.001, 0.01, 0.1, 0.2]
}]
grid_search = GridSearchCV(SVC(), tuned_params, scoring = 'f1', cv = k_fold)
grid_search.fit(bank_attributes_train, bank_labels_train)
gaussian_svm_classifier = grid_search.best_estimator_

print("Gaussian SVM; best score:", grid_search.best_score_)

Gaussian SVM; best score: 0.45760736147055053


In [40]:
print(grid_search.best_params_)

{'C': 20, 'gamma': 0.01}


In [41]:
assert_is_not_none(grid_search)
assert_is_not_none(gaussian_svm_classifier)

### 9. Compare performance on the testing data (1 point)
Now that you've trained all your models, you've got to select the best one. This should be done on the testing data.

Use the appropriate scoring metric to get the testing scores for all your models. Don't forget to pass the **testing**, not the training data. Save all scores.

Choose the best classifier, based on these scores (the one with the highest test score). Of course, this is not enough. We need to look at ROC curves, track performance through other measures, debug the sources of variance in testing results, try more hyperparameters, etc. However, this is enough for an introductory lab :).

Optionally, you can think of combining them into a boosted model but this is out of the scope of this lab.

In [42]:
tree_classifier_score = f1_score(bank_labels_test, tree_classifier.predict(bank_attributes_test))
forest_classifier_score = f1_score(bank_labels_test, forest_classifier.predict(bank_attributes_test))
linear_svm_classifier_score = f1_score(bank_labels_test, linear_svm_classifier.predict(bank_attributes_test))
gaussian_svm_classifier_score = f1_score(bank_labels_test, gaussian_svm_classifier.predict(bank_attributes_test))

print("Testing scores:")
print("Decision tree:", tree_classifier_score)
print("Random forest:", forest_classifier_score)
print("Linear SVM:", linear_svm_classifier_score)
print("Gaussian SVM:", gaussian_svm_classifier_score)

Testing scores:
Decision tree: 0.46212121212121204
Random forest: 0.34862385321100914
Linear SVM: 0.424
Gaussian SVM: 0.4386617100371747


In [45]:
# Replace empty string with "tree", "forest", "linear SVM" or "gaussian SVM"
best_classifier = "tree" # Replace empty string with "tree", "forest", "linear SVM" or "gaussian SVM"

In [46]:
assert_not_equal(best_classifier, "")